In [1]:
import io
import re
import numpy as np
import pandas as pd
import numpy.linalg as lin
import warnings 
from gensim.models import Word2Vec
import pickle
from sklearn.cluster import KMeans
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate
warnings.filterwarnings('ignore')

In [2]:
stopwords=['ଦେଇଛନ୍ତି', 'ଲେଖାଏଁ', 'ଜଣେ', 'ଏହା', 'ତେଣୁ', 'ମିଳିଥାଏ', 'ପାଇଁ', 'ନେଉଛନ୍ତି', 'ଯୋଗୁଁ', 'ଏପର୍ଯ୍ୟନ୍ତ', 'ଏଭଳି', 'କରୁଛି', 'ଓ', 'ଯାଏଁ','ହୋଇଛନ୍ତି', 'କି', 'କରାଯିବା', 'ପରେ', 'ଏହି', 'ଏବଂ', 'ଜଣ', 'ଥିବା','ହୋଇଥିଲା', 'ତେବେ', 'ଆଜି', 'ଜଣଙ୍କ', 'ଏଥି', 'ଗତ', 'ହୋଇଥିଲେ',
       'ଦିନ ଧରି', 'ହେଉଥିବା', 'ଦିନ ତଳେ', 'ଯୋଗେ', 'ବୋଲି', 'ଜଣାପଡ଼ିଛି',
       'ଦ୍ବାରା', 'କରି', 'ଯାଇ', 'ଏ ନେଇ', 'ଚାଲୁ', 'ରହିଛି', 'ତାରିଖ',
       'ମିଳିଛି', 'ବର୍ଷୀୟ', 'ଦିନ ତଳେ', 'ସହ', 'ଆସିଛନ୍ତି', 'ମଧ୍ୟ', 'କେଉଁ',
       'ହୋଇଯାଇଛି', 'ନେଇ ଯାଇଛି', 'କାମ', 'କରିଛି', 'ହେବା', 'ପରେ', 'ଏବେ',
       'ହୋଇଛି', 'ତେବେ', 'ରଖି', 'ନିଆଯାଇଥିବା', 'ପର୍ଯ୍ୟନ୍ତ', 'କରିଛନ୍ତି',
       'ଉପରେ', 'ଦେଲେ', 'ଥର', 'ଏବଂ', 'ଆଉ']
alphabets=['ଅ','ଆ','ଇ','ଈ','ଉ','ଊ','ଋ','ଌ','ଏ','ଐ','ଓ','ଔ','କ','ଖ','ଗ','ଘ','ଙ','ଚ','ଛ','ଜ','ଝ','ଞ','ଟ','ଠ','ଡ','ଢ','ଣ',
 'ତ','ଥ','ଦ','ଧ','ନ','ପ','ଫ','ବ','ଭ','ମ','ଯ','ର','ଲ','ଳ','ଵ','ଶ','ଷ','ସ','ହ','ଡ଼','ଢ଼','ୟ','ୠ','ୡ','ୱ']
matra=['ଁ','ଂ','ଃ','଼','ା','ି','ୀ','ୁ','ୂ','ୃ','ୄ','େ','ୈ','ୋ','ୌ','୍','ୢ','ୣ']

In [3]:
delimiters=["।","?","!"]

In [4]:
cbow=Word2Vec.load('cbow/or-d100-m2-cbow.model')

In [5]:
file=open('sentence.pkl','rb')
verse_embeddings_sentence=pickle.load(file)
file.close()
file=open('max.pkl','rb')
verse_embeddings_max=pickle.load(file)
file.close()
file=open('mean.pkl','rb')
verse_embeddings_mean=pickle.load(file)
file.close()

In [6]:
file=open('whole.pkl','rb')
verse_embeddings_whole=pickle.load(file)
file.close()

In [7]:
def o2h(text):
    return transliterate(text, sanscript.ORIYA, sanscript.DEVANAGARI)

In [8]:
def remove_special_characters(text):
    string=''
    for i in text:
        if i in alphabets+matra+delimiters:
            string=string+i
        else:
            string=string+" "
    return " ".join(string.split())
def sent_tokenize(text):
    text=remove_special_characters(text)
    regexPattern = '|'.join(map(re.escape, delimiters))
    sentences=re.split(regexPattern,text)
    return [sent.strip() for sent in sentences if len(sent.strip())>0]

In [9]:
def word_embedding(word):
    try:
        return cbow.wv[o2h(word)]
    except:
        return np.array([0.0]*100)

In [10]:
def cosine_similarity(a,b):
    try:
        return a.dot(b)/(lin.norm(a)*lin.norm(b))
    except:
        return -1.0

In [11]:
def ed(a,b):
    return lin.norm(a-b)

In [12]:
def sentence_embedding(sentence):
    sentence=remove_special_characters(sentence)
    words=sentence.split()
    words=[word  for word in words if word not in stopwords]
    if len(words)>0:
        sentence_embedding=[word_embedding(word) for word in words]
        return np.array(list(map(lambda x: sum(x)/len(x), zip(*sentence_embedding))))
    return np.array([0.0]*100)

In [13]:
data=pd.read_csv('../odia.csv')

In [14]:
query='ସବୁଠାରୁ ଉପଯୁକ୍ତ କାର୍ଯ୍ୟ ପଦ୍ଧତି ବିଷୟରେ ଅନୁମାନ କରିବାକୁ ଯାଇ'
query_embedding=sentence_embedding(query)
query_embedding_list=query_embedding.reshape(1,-1).astype(np.float64)

# Max Strategy

## Cosine Similarity

In [16]:
def max_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_max]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [17]:
verse_index=max_cosine(query_embedding)

In [18]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
43,2,4,अर्जुन उवाच\n\nकथं भीष्ममहं संख्ये द्रोणं च मध...,କର୍ମ କରିବା ପାଇଁ ଶ୍ରୀକୃଷ୍ଣଙ୍କ ଆହ୍ୱାନର ପ୍ରତିକ୍ରି...
678,18,64,सर्वगुह्यतमं भूयः श्रृणु मे परमं वचः।इष्टोऽसि ...,"ଜଣେ ଶିକ୍ଷକ ଅତ୍ୟନ୍ତ ଗୁଢ଼ ଜ୍ଞାନ ଜାଣିଥାଇ ପାରନ୍ତି, ..."
363,9,33,किं पुनर्ब्राह्मणाः पुण्या भक्ता राजर्षयस्तथा।...,ଯଦି ଅତ୍ୟନ୍ତ ପାପୀମାନେ ମଧ୍ୟ ଭକ୍ତି ମାର୍ଗରେ ସଫଳତା ...
615,18,1,अर्जुन उवाच\n\nसंन्यासस्य महाबाहो तत्त्वमिच्छा...,"ଅର୍ଜୁନ ଶ୍ରୀକୃଷ୍ଣଙ୍କୁ “କେଶି-ନିଷୂଦନ”, ଅର୍ଥାତ୍ “କ..."
72,2,33,अथ चैत्त्वमिमं धर्म्यं संग्रामं न करिष्यसि।\n\...,ଯଦି ଜଣେ ଯୋଦ୍ଧା ଯୁଦ୍ଧକ୍ଷେତ୍ରରେ ଅହିଂସା ଆଚରଣ କରନ୍...
329,8,27,नैते सृती पार्थ जानन्योगी मुह्यति कश्चन।\n\nतस...,"ଯୋଗୀମାନେ ସେହି ପ୍ରକାର ସାଧକ ଅଟନ୍ତି, ଯେଉଁମାନେ ନିଜ..."
680,18,66,सर्वधर्मान्परित्यज्य मामेकं शरणं व्रज।अहं त्वा...,"ଏ ପର୍ଯ୍ୟନ୍ତ, ଶ୍ରୀକୃଷ୍ଣ ଅର୍ଜୁନଙ୍କୁ ଏକ ସଙ୍ଗେ ଦୁଇ..."
645,18,31,यया धर्ममधर्मं च कार्यं चाकार्यमेव च।अयथावत्प्...,ବ୍ୟକ୍ତିଗତ ଆସକ୍ତି ଦ୍ୱାରା ରାଜସିକ ବୁଦ୍ଧି ମିଶ୍ରିତ ...
193,4,39,श्रद्धावाँल्लभते ज्ञानं तत्परः संयतेन्द्रियः।\...,"ଶ୍ରୀକୃଷ୍ଣ ବର୍ତ୍ତମାନ, ଜ୍ଞାନର ପରିପ୍ରେକ୍ଷୀରେ, ଶ୍ର..."
116,3,5,न हि कश्िचत्क्षणमपि जातु तिष्ठत्यकर्मकृत्।\n\n...,କେତେକ ବ୍ୟକ୍ତି ଭାବନ୍ତି ଯେ କର୍ମ କେବଳ ବୃତ୍ତିଗତ କର...


## Euclidean Distance

In [19]:
def max_ed(query_embedding,counter=10):
    scoring=[ed(query_embedding,i) for i in verse_embeddings_max]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [20]:
verse_index=max_ed(query_embedding)

In [21]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
43,2,4,अर्जुन उवाच\n\nकथं भीष्ममहं संख्ये द्रोणं च मध...,କର୍ମ କରିବା ପାଇଁ ଶ୍ରୀକୃଷ୍ଣଙ୍କ ଆହ୍ୱାନର ପ୍ରତିକ୍ରି...
678,18,64,सर्वगुह्यतमं भूयः श्रृणु मे परमं वचः।इष्टोऽसि ...,"ଜଣେ ଶିକ୍ଷକ ଅତ୍ୟନ୍ତ ଗୁଢ଼ ଜ୍ଞାନ ଜାଣିଥାଇ ପାରନ୍ତି, ..."
615,18,1,अर्जुन उवाच\n\nसंन्यासस्य महाबाहो तत्त्वमिच्छा...,"ଅର୍ଜୁନ ଶ୍ରୀକୃଷ୍ଣଙ୍କୁ “କେଶି-ନିଷୂଦନ”, ଅର୍ଥାତ୍ “କ..."
680,18,66,सर्वधर्मान्परित्यज्य मामेकं शरणं व्रज।अहं त्वा...,"ଏ ପର୍ଯ୍ୟନ୍ତ, ଶ୍ରୀକୃଷ୍ଣ ଅର୍ଜୁନଙ୍କୁ ଏକ ସଙ୍ଗେ ଦୁଇ..."
329,8,27,नैते सृती पार्थ जानन्योगी मुह्यति कश्चन।\n\nतस...,"ଯୋଗୀମାନେ ସେହି ପ୍ରକାର ସାଧକ ଅଟନ୍ତି, ଯେଉଁମାନେ ନିଜ..."
193,4,39,श्रद्धावाँल्लभते ज्ञानं तत्परः संयतेन्द्रियः।\...,"ଶ୍ରୀକୃଷ୍ଣ ବର୍ତ୍ତମାନ, ଜ୍ଞାନର ପରିପ୍ରେକ୍ଷୀରେ, ଶ୍ର..."
237,6,12,तत्रैकाग्रं मनः कृत्वा यतचित्तेन्द्रियक्रियः।\...,ଧ୍ୟାନରେ ବସିବାର ଆସନ ବିଷୟରେ ବର୍ଣ୍ଣନା କରି ବର୍ତ୍ତମ...
238,6,13,समं कायशिरोग्रीवं धारयन्नचलं स्थिरः।\n\nसंप्रे...,ଧ୍ୟାନରେ ବସିବାର ଆସନ ବିଷୟରେ ବର୍ଣ୍ଣନା କରି ବର୍ତ୍ତମ...
116,3,5,न हि कश्िचत्क्षणमपि जातु तिष्ठत्यकर्मकृत्।\n\n...,କେତେକ ବ୍ୟକ୍ତି ଭାବନ୍ତି ଯେ କର୍ମ କେବଳ ବୃତ୍ତିଗତ କର...
44,2,5,गुरूनहत्वा हि महानुभावान्\n\nश्रेयो भोक्तुं भै...,ଏପରି ଯୁକ୍ତି କରାଯାଇ ପାରେ ଯେ ଅର୍ଜୁନ ନିଜର ଜୀବନ ନି...


# Mean Pooling

## Cosine Similarity

In [22]:
def mean_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_mean]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [23]:
verse_index=mean_cosine(query_embedding)

In [24]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
644,18,30,प्रवृत्तिं च निवृत्तिं च कार्याकार्ये भयाभये।ब...,କୌଣସି ନିର୍ଣ୍ଣୟ ଗ୍ରହଣ କରିବା ପାଇଁ ଆମ୍ଭେମାନେ ସର୍ବ...
294,7,22,स तया श्रद्धया युक्तस्तस्याराधनमीहते।\n\nलभते ...,ଲଭତେ ଅର୍ଥ “ସେମାନେ ପ୍ରାପ୍ତ କରନ୍ତି ।” ଦେବତାମାନଙ୍...
573,16,11,चिन्तामपरिमेयां च प्रलयान्तामुपाश्रिताः।कामोपभ...,ବସ୍ତୁବାଦୀମାନେ ଅଧିକାଂଶତଃ ଏହି କାରଣରୁ ଆଧ୍ୟାତ୍ମିକ ...
364,9,34,मन्मना भव मद्भक्तो मद्याजी मां नमस्कुरु।\n\nमा...,ପୂରା ଅଧ୍ୟାୟରେ ଭକ୍ତି ଏବଂ ଭକ୍ତି ମାର୍ଗ ଉପରେ ଗୁରୁତ...
471,12,10,अभ्यासेऽप्यसमर्थोऽसि मत्कर्मपरमो भव।मदर्थमपि क...,ଭଗବାନଙ୍କର ସ୍ମରଣ କରିବା ପାଇଁ ଉପଦେଶ ଦେବା ତାହାକୁ କ...
71,2,32,यदृच्छया चोपपन्नं स्वर्गद्वारमपावृतम्।\n\nसुखि...,ସମାଜକୁ ସୁରକ୍ଷା ଦେବା ପାଇଁ କ୍ଷତ୍ରିୟମାନଙ୍କର ଆବଶ୍ୟ...
310,8,8,अभ्यासयोगयुक्तेन चेतसा नान्यगामिना।\n\nपरमं पु...,"ଭଗବଦ୍ ଗୀତାରେ ଅନେକ ସ୍ଥାନରେ, ଭଗବାନଙ୍କର ସ୍ମରଣ ସର୍..."
258,6,33,अर्जुन उवाच\n\nयोऽयं योगस्त्वया प्रोक्तः साम्य...,ଅର୍ଜୁନ ଏହି ଶ୍ଲୋକଟି ଯୋଽୟଂ ଶବ୍ଦରୁ ଆରମ୍ଭ କରିଛନ୍ତି...
72,2,33,अथ चैत्त्वमिमं धर्म्यं संग्रामं न करिष्यसि।\n\...,ଯଦି ଜଣେ ଯୋଦ୍ଧା ଯୁଦ୍ଧକ୍ଷେତ୍ରରେ ଅହିଂସା ଆଚରଣ କରନ୍...
238,6,13,समं कायशिरोग्रीवं धारयन्नचलं स्थिरः।\n\nसंप्रे...,ଧ୍ୟାନରେ ବସିବାର ଆସନ ବିଷୟରେ ବର୍ଣ୍ଣନା କରି ବର୍ତ୍ତମ...


## Euclidean Distance

In [25]:
def mean_ed(query_embedding,counter=10):
    scoring=[ed(query_embedding,i) for i in verse_embeddings_mean]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [26]:
verse_index=mean_ed(query_embedding)

In [27]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
294,7,22,स तया श्रद्धया युक्तस्तस्याराधनमीहते।\n\nलभते ...,ଲଭତେ ଅର୍ଥ “ସେମାନେ ପ୍ରାପ୍ତ କରନ୍ତି ।” ଦେବତାମାନଙ୍...
340,9,10,मयाऽध्यक्षेण प्रकृतिः सूयते सचराचरम्।\n\nहेतुन...,"ପୂର୍ବ ଶ୍ଳୋକରେ ବର୍ଣ୍ଣନା କରାଯାଇଥିବା ପ୍ରକାରେ, ଭଗବ..."
471,12,10,अभ्यासेऽप्यसमर्थोऽसि मत्कर्मपरमो भव।मदर्थमपि क...,ଭଗବାନଙ୍କର ସ୍ମରଣ କରିବା ପାଇଁ ଉପଦେଶ ଦେବା ତାହାକୁ କ...
644,18,30,प्रवृत्तिं च निवृत्तिं च कार्याकार्ये भयाभये।ब...,କୌଣସି ନିର୍ଣ୍ଣୟ ଗ୍ରହଣ କରିବା ପାଇଁ ଆମ୍ଭେମାନେ ସର୍ବ...
185,4,31,यज्ञशिष्टामृतभुजो यान्ति ब्रह्म सनातनम्।\n\nना...,ପୂର୍ବରୁ ଯଜ୍ଞର ରହସ୍ୟ ଭାବରେ ଯାହା ବର୍ଣ୍ଣନା କରାଯାଇ...
439,11,33,तस्मात्त्वमुत्तिष्ठ यशो लभस्व\n\nजित्वा शत्रून...,ଶ୍ରୀକୃଷ୍ଣ ଅର୍ଜୁନଙ୍କ ସମ୍ମୁଖରେ ତାଙ୍କର ଅଭିଳାଷ ପ୍ର...
625,18,11,न हि देहभृता शक्यं त्यक्तुं कर्माण्यशेषतः।यस्त...,ଏଠାରେ ଯୁକ୍ତି କରାଯାଇପାରେ ଯେ କର୍ମଫଳ ତ୍ୟାଗ କରିବା ...
672,18,58,मच्चित्तः सर्वदुर्गाणि मत्प्रसादात्तरिष्यसि।अथ...,"ପୂର୍ବ ଶ୍ଳୋକରେ କ’ଣ କରଣୀୟ ତାହାର ଉପଦେଶ ଦେବା ପରେ, ..."
169,4,15,एवं ज्ञात्वा कृतं कर्म पूर्वैरपि मुमुक्षुभिः।\...,ଯେଉଁ ମହାପୁରୁଷମାନେ ଭଗବାନଙ୍କୁ ପ୍ରାପ୍ତ କରିବାକୁ ଇଚ...
364,9,34,मन्मना भव मद्भक्तो मद्याजी मां नमस्कुरु।\n\nमा...,ପୂରା ଅଧ୍ୟାୟରେ ଭକ୍ତି ଏବଂ ଭକ୍ତି ମାର୍ଗ ଉପରେ ଗୁରୁତ...


# Line Matching

## Cosine Similarity

In [28]:
def line_cosine(query_embeddings,counter=10):
    scoring=[]
    for embeddings in verse_embeddings_sentence:
        score=[cosine_similarity(query_embedding,embed) for embed in embeddings]
        scoring.append(max(score))
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [29]:
verse_index=line_cosine(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
45,2,6,न चैतद्विद्मः कतरन्नो गरीयो\n\nयद्वा जयेम यदि ...,ସବୁଠାରୁ ଉପଯୁକ୍ତ କାର୍ଯ୍ୟ ପଦ୍ଧତି ବିଷୟରେ ଅନୁମାନ କ...
113,3,2,व्यामिश्रेणेव वाक्येन बुद्धिं मोहयसीव मे।\n\nत...,"ପ୍ରଥମ ଅଧ୍ୟାୟର ପୃଷ୍ଠଭୂମି, ଅର୍ଜୁନଙ୍କର ଦୁଃଖ ଏବଂ ପ..."
112,3,1,अर्जुन उवाच\n\nज्यायसी चेत्कर्मणस्ते मता बुद्ध...,"ପ୍ରଥମ ଅଧ୍ୟାୟର ପୃଷ୍ଠଭୂମି, ଅର୍ଜୁନଙ୍କର ଦୁଃଖ ଏବଂ ପ..."
162,4,8,परित्राणाय साधूनां विनाशाय च दुष्कृताम्।\n\nधर...,"ପୂର୍ବ ଶ୍ଲୋକରେ, ଭଗବାନ ସଂସାରରେ ଅବତାର ଗ୍ରହଣ କରନ୍ତ..."
143,3,32,ये त्वेतदभ्यसूयन्तो नानुतिष्ठन्ति मे मतम्।\n\n...,ଶ୍ରୀକୃଷ୍ଣଙ୍କ ଦ୍ୱାରା ପ୍ରଦାନ କରାଯାଇ ଥିବା ଉପଦେଶ ନ...
632,18,18,ज्ञानं ज्ञेयं परिज्ञाता त्रिविधा कर्मचोदना।करण...,"କର୍ମ ବିଜ୍ଞାନର ପ୍ରଣାଳୀବଦ୍ଧ ବିଶ୍ଳେଷଣ ଦ୍ୱାରା, ଶ୍ର..."
331,9,1,श्री भगवानुवाच\n\nइदं तु ते गुह्यतमं प्रवक्ष्य...,ବିଷୟ ପ୍ରାରମ୍ଭରୁ ହିଁ ଶ୍ରୀକୃଷ୍ଣ ଏହି ଉପଦେଶ ଶୁଣିବା...
643,18,29,बुद्धेर्भेदं धृतेश्चैव गुणतस्त्रिविधं श्रृणु।प...,"ପୂର୍ବବର୍ତ୍ତୀ ନଅଗୋଟି ଶ୍ଳୋକରେ, ଶ୍ରୀକୃଷ୍ଣ କର୍ମର ଉ..."
535,14,20,गुणानेतानतीत्य त्रीन्देही देहसमुद्भवान्।जन्ममृ...,"ଆମେ ଯଦି ଗୋଟିଏ ଅପରିଷ୍କାର କୂଅରୁ ଜଳପାନ କରୁ, ଆମର ଉ..."
487,13,7,इच्छा द्वेषः सुखं दुःखं सङ्घातश्चेतनाधृतिः।एतत...,ଶ୍ରୀକୃଷ୍ଣ ବର୍ତ୍ତମାନ କ୍ଷେତ୍ରର ବୈଶିଷ୍ଟ୍ୟ ଏବଂ ଏହା...


## Euclidean Distance

In [30]:
def line_ed(query_embeddings,counter=10):
    scoring=[]
    for embeddings in verse_embeddings_sentence:
        score=[ed(query_embedding,embed) for embed in embeddings]
        scoring.append(max(score))
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [31]:
verse_index=line_ed(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
364,9,34,मन्मना भव मद्भक्तो मद्याजी मां नमस्कुरु।\n\nमा...,ପୂରା ଅଧ୍ୟାୟରେ ଭକ୍ତି ଏବଂ ଭକ୍ତି ମାର୍ଗ ଉପରେ ଗୁରୁତ...
512,13,32,अनादित्वान्निर्गुणत्वात्परमात्मायमव्ययः।शरीरस्...,"ଭଗବାନ ସମସ୍ତ ପ୍ରାଣୀଙ୍କ ହୃଦୟରେ, ପରମାତ୍ମା ରୂପରେ ନ..."
455,11,49,मा ते व्यथा मा च विमूढभावो\n\nदृष्ट्वा रूपं घो...,ଶ୍ରୀକୃଷ୍ଣ ଅର୍ଜୁନଙ୍କୁ କ୍ରମାଗତ ଭାବରେ ଆଶ୍ୱାସନା ଦେ...
294,7,22,स तया श्रद्धया युक्तस्तस्याराधनमीहते।\n\nलभते ...,ଲଭତେ ଅର୍ଥ “ସେମାନେ ପ୍ରାପ୍ତ କରନ୍ତି ।” ଦେବତାମାନଙ୍...
365,10,1,श्री भगवानुवाच\n\nभूय एव महाबाहो श्रृणु मे परम...,ତାଙ୍କର ମହିମା ଶ୍ରବଣ କରିବାରେ ଅର୍ଜୁନଙ୍କର ଆଗ୍ରହ ଦେ...
76,2,37,हतो वा प्राप्स्यसि स्वर्गं जित्वा वा भोक्ष्यसे...,୨.୩୧ ଶ୍ଳୋକ ଠାରୁ ଆରମ୍ଭ କରି ଶ୍ରୀକୃଷ୍ଣ ଏ ପର୍ଯ୍ୟନ୍...
672,18,58,मच्चित्तः सर्वदुर्गाणि मत्प्रसादात्तरिष्यसि।अथ...,"ପୂର୍ବ ଶ୍ଳୋକରେ କ’ଣ କରଣୀୟ ତାହାର ଉପଦେଶ ଦେବା ପରେ, ..."
603,17,17,श्रद्धया परया तप्तं तपस्तत्ित्रविधं नरैः।अफलाक...,"ଶରୀର, ମନ ଓ ବାଣୀର ସଂଯମ ବିଷୟରେ ବର୍ଣ୍ଣନା କରିବା ପର..."
691,18,77,तच्च संस्मृत्य संस्मृत्य रूपमत्यद्भुतं हरेः।\n...,ଅର୍ଜୁନ ଭଗବାନଙ୍କର ବିଶ୍ୱରୂପ ଦର୍ଶନ କରିବାର ସୌଭାଗ୍ୟ...
625,18,11,न हि देहभृता शक्यं त्यक्तुं कर्माण्यशेषतः।यस्त...,ଏଠାରେ ଯୁକ୍ତି କରାଯାଇପାରେ ଯେ କର୍ମଫଳ ତ୍ୟାଗ କରିବା ...


# Whole Part

## Cosine Similarity

In [32]:
def whole_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_whole]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [33]:
verse_index=whole_cosine(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
258,6,33,अर्जुन उवाच\n\nयोऽयं योगस्त्वया प्रोक्तः साम्य...,ଅର୍ଜୁନ ଏହି ଶ୍ଲୋକଟି ଯୋଽୟଂ ଶବ୍ଦରୁ ଆରମ୍ଭ କରିଛନ୍ତି...
137,3,26,न बुद्धिभेदं जनयेदज्ञानां कर्मसङ्गिनाम्।\n\nजो...,"ମହାପୁରୁଷମାନଙ୍କର ଅଧିକ ଦାୟୀତ୍ୱ ରହିଥାଏ, କାରଣ ସାଧା..."
644,18,30,प्रवृत्तिं च निवृत्तिं च कार्याकार्ये भयाभये।ब...,କୌଣସି ନିର୍ଣ୍ଣୟ ଗ୍ରହଣ କରିବା ପାଇଁ ଆମ୍ଭେମାନେ ସର୍ବ...
71,2,32,यदृच्छया चोपपन्नं स्वर्गद्वारमपावृतम्।\n\nसुखि...,ସମାଜକୁ ସୁରକ୍ଷା ଦେବା ପାଇଁ କ୍ଷତ୍ରିୟମାନଙ୍କର ଆବଶ୍ୟ...
364,9,34,मन्मना भव मद्भक्तो मद्याजी मां नमस्कुरु।\n\nमा...,ପୂରା ଅଧ୍ୟାୟରେ ଭକ୍ତି ଏବଂ ଭକ୍ତି ମାର୍ଗ ଉପରେ ଗୁରୁତ...
261,6,36,असंयतात्मना योगो दुष्प्राप इति मे मतिः।\n\nवश्...,"ପରମ ପୁରୁଷ ଶ୍ରୀକୃଷ୍ଣ, ବତ୍ତର୍ମାନ ମନ ନିୟନ୍ତ୍ରଣ ଏବ..."
471,12,10,अभ्यासेऽप्यसमर्थोऽसि मत्कर्मपरमो भव।मदर्थमपि क...,ଭଗବାନଙ୍କର ସ୍ମରଣ କରିବା ପାଇଁ ଉପଦେଶ ଦେବା ତାହାକୁ କ...
439,11,33,तस्मात्त्वमुत्तिष्ठ यशो लभस्व\n\nजित्वा शत्रून...,ଶ୍ରୀକୃଷ୍ଣ ଅର୍ଜୁନଙ୍କ ସମ୍ମୁଖରେ ତାଙ୍କର ଅଭିଳାଷ ପ୍ର...
238,6,13,समं कायशिरोग्रीवं धारयन्नचलं स्थिरः।\n\nसंप्रे...,ଧ୍ୟାନରେ ବସିବାର ଆସନ ବିଷୟରେ ବର୍ଣ୍ଣନା କରି ବର୍ତ୍ତମ...
237,6,12,तत्रैकाग्रं मनः कृत्वा यतचित्तेन्द्रियक्रियः।\...,ଧ୍ୟାନରେ ବସିବାର ଆସନ ବିଷୟରେ ବର୍ଣ୍ଣନା କରି ବର୍ତ୍ତମ...


## Euclidean Distance

In [34]:
def whole_ed(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_whole]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [35]:
verse_index=whole_ed(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
391,10,27,उच्चैःश्रवसमश्वानां विद्धि माममृतोद्भवम्।\n\nऐ...,ସବୁ ବିଭାଗର ଶ୍ରେଷ୍ଠମାନଙ୍କୁ ଶ୍ରୀକୃଷ୍ଣ ତାଙ୍କର ଐଶ୍...
689,18,75,व्यासप्रसादाच्छ्रुतवानेतद्गुह्यमहं परम्।योगं य...,"ଶ୍ରୀକୃଷ୍ଣ ଦ୍ୱୈପାୟନ ବ୍ୟାସଦେବ, ଯିଏ ମହର୍ଷି ବେଦବ୍ୟ..."
401,10,37,वृष्णीनां वासुदेवोऽस्मि पाण्डवानां धनंजयः।\n\n...,ବୃଷ୍ଣୀବଂଶରେ ବସୁଦେବଙ୍କ ପୁତ୍ର ରୂପରେ ଶ୍ରୀକୃଷ୍ଣ ପୃ...
394,10,30,प्रह्लादश्चास्मि दैत्यानां कालः कलयतामहम्।\n\n...,ପରାକ୍ରମଶାଳୀ ଦୈତ୍ୟରାଜ ହିରଣ୍ୟ-କଶିପୁଙ୍କ ପୁତ୍ର ଭାବ...
412,11,6,पश्यादित्यान्वसून्रुद्रानश्िवनौ मरुतस्तथा।\n\n...,ଭଗବାନଙ୍କର ବିଶ୍ୱରୂପରେ ପୃଥିବୀ ପୃଷ୍ଠର ସମସ୍ତ ଐଶ୍ଚର...
397,10,33,अक्षराणामकारोऽस्मि द्वन्द्वः सामासिकस्य च।\n\n...,"ସଂସ୍କୃତ ଭାଷାରେ, ଯେ କୌଣସି ଅର୍ଦ୍ଧ ଅକ୍ଷର ସହିତ ‘ଅ’..."
29,1,34,आचार्याः पितरः पुत्रास्तथैव च पितामहाः।\n\nमात...,ଦ୍ରୋଣାଚାର୍ଯ୍ୟ ଏବଂ କୃପାଚାର୍ଯ୍ୟ ଅର୍ଜୁନଙ୍କ ଗୁରୁ ଥ...
30,1,35,एतान्न हन्तुमिच्छामि घ्नतोऽपि मधुसूदन।\n\nअपि ...,ଦ୍ରୋଣାଚାର୍ଯ୍ୟ ଏବଂ କୃପାଚାର୍ଯ୍ୟ ଅର୍ଜୁନଙ୍କ ଗୁରୁ ଥ...
484,13,4,तत्क्षेत्रं यच्च यादृक् च यद्विकारि यतश्च यत्।...,ବର୍ତ୍ତମାନ ଶ୍ରୀକୃଷ୍ଣ ନିଜେ ଅନେକ ପ୍ରଶ୍ନ ଉପସ୍ଥାପନ ...
500,13,20,प्रकृतिं पुरुषं चैव विद्ध्यनादी उभावपि।विकारां...,ଭୌତିକ ଗୁଣକୁ ମାୟା ବା ପ୍ରକୃତି କୁହାଯାଏ । ଏହା ଭଗବା...


# Clustering

## Max Pooling

In [36]:
kmeans_max = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_max)

In [37]:
def kmeans_max_cluster(query_embedding_list):
    label=kmeans_max.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_max.labels_[i]==label]
    return verse_index

In [38]:
verse_index=kmeans_max_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
46,2,7,कार्पण्यदोषोपहतस्वभावः\n\nपृच्छामि त्वां धर्मस...,"ଭଗବଦ୍ ଗୀତାରେ ଏହା ଏକ ଗୁରୁତ୍ୱପୂର୍ଣ୍ଣ ମୁହୂର୍ତ୍ତ, ..."
65,2,26,अथ चैनं नित्यजातं नित्यं वा मन्यसे मृतम्।\n\nत...,ଶ୍ରୀକୃଷ୍ଣ ଏଠାରେ ଅଥ ଶବ୍ଦ ପ୍ରୟୋଗ କରି ସୂଚିତ କରୁଛନ...
116,3,5,न हि कश्िचत्क्षणमपि जातु तिष्ठत्यकर्मकृत्।\n\n...,କେତେକ ବ୍ୟକ୍ତି ଭାବନ୍ତି ଯେ କର୍ମ କେବଳ ବୃତ୍ତିଗତ କର...
168,4,14,न मां कर्माणि लिम्पन्ति न मे कर्मफले स्पृहा।\n...,ଭଗବାନ ସର୍ବଥା ଶୁଦ୍ଧ ଅଟନ୍ତି । ସେ ଯାହାବି କରନ୍ତି ତ...
179,4,25,दैवमेवापरे यज्ञं योगिनः पर्युपासते।\n\nब्रह्मा...,"ଯଜ୍ଞ, ଦିବ୍ୟ ଚେତନାଯୁକ୍ତ ହୋଇ ଭଗବାନଙ୍କ ପ୍ରତି ସମର୍..."
207,5,11,कायेन मनसा बुद्ध्या केवलैरिन्द्रियैरपि।\n\nयोग...,ଯୋଗୀମାନେ ଏହା ହୃଦୟଙ୍ଗମ କରିଥାନ୍ତି ଯେ ଆନନ୍ଦ ପ୍ରାପ...
248,6,23,तं विद्याद् दुःखसंयोगवियोगं योगसंज्ञितम्।\n\nस...,ଏହି ଭୌତିକ ସଂସାର ମାୟାର ଜଗତ ଅଟେ । ଏହାକୁ ଶ୍ରୀକୃଷ୍...
255,6,30,यो मां पश्यति सर्वत्र सर्वं च मयि पश्यति।\n\nत...,"ଭଗବାନଙ୍କୁ ହରାଇବା ଅର୍ଥ, ମନ ତାଙ୍କୁ ଛାଡି ଅନ୍ୟତ୍ର ..."
265,6,40,श्री भगवानुवाच\n\nपार्थ नैवेह नामुत्र विनाशस्त...,"‘ତାତ’ ଏକ ସ୍ନେହପୂର୍ଣ୍ଣ ସମ୍ଭାଷଣ, ଯାହାର ଶାବ୍ଦିକ ଅ..."
273,7,1,श्री भगवानुवाच\n\nमय्यासक्तमनाः पार्थ योगं युञ...,୬ଷ୍ଠ ଅଧ୍ୟାୟରେ ଶେଷରେ ଶ୍ରୀକୃଷ୍ଣ ଘୋଷଣା କରିଥିଲେ ଯେ...


## Mean Pooling

In [39]:
kmeans_mean = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_mean)

In [40]:
def kmeans_mean_cluster(query_embedding_list):
    label=kmeans_mean.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_mean.labels_[i]==label]
    return verse_index

In [41]:
verse_index=kmeans_mean_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
340,9,10,मयाऽध्यक्षेण प्रकृतिः सूयते सचराचरम्।\n\nहेतुन...,"ପୂର୍ବ ଶ୍ଳୋକରେ ବର୍ଣ୍ଣନା କରାଯାଇଥିବା ପ୍ରକାରେ, ଭଗବ..."


## Whole

In [42]:
kmeans_whole = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_whole)

In [43]:
def kmeans_whole_cluster(query_embedding_list):
    label=kmeans_whole.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_whole.labels_[i]==label]
    return verse_index

In [44]:
verse_index=kmeans_whole_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
80,2,41,व्यवसायात्मिका बुद्धिरेकेह कुरुनन्दन।\n\nबहुशा...,ଆସକ୍ତି ମନର ଏକ ପ୍ରକ୍ରିୟା ଅଟେ । ମନ ଯାହା ପ୍ରତି ଆସ...
83,2,44,भोगैश्वर्यप्रसक्तानां तयापहृतचेतसाम्।\n\nव्यवस...,"ଯେଉଁମାନଙ୍କର ମନ ଇନ୍ଦ୍ରିୟ ସୁଖରେ ଆସକ୍ତ ରହିଥାଏ, ସେ..."
235,6,10,योगी युञ्जीत सततमात्मानं रहसि स्थितः।\n\nएकाकी...,ଯୋଗର ସ୍ଥିତିରେ ଉପନୀତ ବ୍ୟକ୍ତିଙ୍କର ଲକ୍ଷଣ ବର୍ଣ୍ଣନା...
251,6,26,यतो यतो निश्चरति मनश्चञ्चलमस्थिरम्।\n\nततस्ततो...,ଧ୍ୟାନରେ ସଫଳତା ଗୋଟିଏ ଦିନରେ ମିଳି ନ ଥାଏ । ସଫଳତାର ...
317,8,15,मामुपेत्य पुनर्जन्म दुःखालयमशाश्वतम्।\n\nनाप्न...,ଭଗବତ୍ ପ୍ରାପ୍ତିର ଫଳ କ’ଣ? ଯେଉଁମାନେ ଭଗବତ୍ ପ୍ରାପ୍ତ...
526,14,11,सर्वद्वारेषु देहेऽस्मिन्प्रकाश उपजायते।ज्ञानं ...,ବ୍ୟକ୍ତିର ବିଚାରକୁ ଏହି ତିନିଗୁଣ କିପରି ପ୍ରଭାବିତ କର...
527,14,12,लोभः प्रवृत्तिरारम्भः कर्मणामशमः स्पृहा।रजस्ये...,ବ୍ୟକ୍ତିର ବିଚାରକୁ ଏହି ତିନିଗୁଣ କିପରି ପ୍ରଭାବିତ କର...
528,14,13,अप्रकाशोऽप्रवृत्तिश्च प्रमादो मोह एव च।तमस्येत...,ବ୍ୟକ୍ତିର ବିଚାରକୁ ଏହି ତିନିଗୁଣ କିପରି ପ୍ରଭାବିତ କର...
603,17,17,श्रद्धया परया तप्तं तपस्तत्ित्रविधं नरैः।अफलाक...,"ଶରୀର, ମନ ଓ ବାଣୀର ସଂଯମ ବିଷୟରେ ବର୍ଣ୍ଣନା କରିବା ପର..."
607,17,21,यत्तु प्रत्युपकारार्थं फलमुद्दिश्य वा पुनः।दीय...,"କେହି ଦାନ ମାଗି ନ ଥିବା ସତ୍ତ୍ୱେ, ସ୍ୱେଚ୍ଛାକୃତ ଭାବର..."
